In [30]:
# import dependencies 
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np
from config import pg_password
from sqlalchemy import create_engine

## Read in data using pd.read_csv()

In [31]:
# Read in csv
covid_data_df = pd.read_csv(r"us-counties.csv")
covid_data_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [32]:
# Read in csv with Geolocation data of each county in US
geo_ca_df = pd.read_csv(r"us_cities_with_longitude,latitude_data/us-county-boundaries.csv")
geo_ca_df.head()

,Geo Point,Geo Shape,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,STUSAB,LSAD,...,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,STATE_NAME,COUNTYFP NOZERO
0,"36.5111268089, -117.410789686","{""type"": ""Polygon"", ""coordinates"": [[[-118.337...",6,27,1804637,6027,Inyo,Inyo County,CA,6,...,NaN,NaN,NaN,A,26410680788,76915743,36.561977,-117.403927,California,27
1,"37.9233794475, -121.951073113","{""type"": ""Polygon"", ""coordinates"": [[[-122.267...",6,13,1675903,6013,Contra Costa,Contra Costa County,CA,6,...,488.0,41860.0,36084.0,A,1857229826,225285866,37.919479,-121.951543,California,13
2,"38.5972506332, -119.82065303","{""type"": ""Polygon"", ""coordinates"": [[[-120.073...",6,3,1675840,6003,Alpine,Alpine County,CA,6,...,NaN,NaN,NaN,A,1912292608,12557304,38.621783,-119.798352,California,3
3,"40.7050090829, -123.915819446","{""type"": ""Polygon"", ""coordinates"": [[[-124.281...",6,23,1681908,6023,Humboldt,Humboldt County,CA,6,...,NaN,21700.0,NaN,A,9241426488,1253864712,40.706655,-123.926176,California,23
4,"39.0345595213, -121.694849665","{""type"": ""Polygon"", ""coordinates"": [[[-121.928...",6,101,277315,6101,Sutter,Sutter County,CA,6,...,472.0,49700.0,NaN,A,1560546528,15240591,39.036190,-121.703940,California,101


## Check DataFrame values
- Total date values
- Earliest date
- Latest date
    

In [33]:
# Identify total date nunique() values
covid_data_df['date'].nunique()

369

In [34]:
# Identify min() date range
covid_data_df['date'].min()

'2020-01-21'

In [35]:
# Identify max() date range
covid_data_df['date'].max()

'2021-01-23'

## Clean and Transform Data
- Format date from string to_datetime
- Gather data for only California
- Gather data for only 2020

In [36]:
# convert date column from string to datetime
covid_data_df['date'] = pd.to_datetime(covid_data_df['date'], format='%Y/%m/%d')
covid_data_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [37]:
# Create dataFrame for only California 
covid_cali_df = covid_data_df.loc[covid_data_df["state"] == "California"] 
covid_cali_df.head()

,date,county,state,fips,cases,deaths
5,2020-01-25,Orange,California,6059.0,1,0.0
9,2020-01-26,Los Angeles,California,6037.0,1,0.0
10,2020-01-26,Orange,California,6059.0,1,0.0
14,2020-01-27,Los Angeles,California,6037.0,1,0.0
15,2020-01-27,Orange,California,6059.0,1,0.0


In [38]:
# Get county names 
covid_cali_df.county.unique()

array(['Orange', 'Los Angeles', 'Santa Clara', 'San Francisco',
       'San Diego', 'Humboldt', 'Sacramento', 'Solano', 'Marin', 'Napa',
       'Sonoma', 'Alameda', 'Placer', 'San Mateo', 'Contra Costa', 'Yolo',
       'Fresno', 'Madera', 'Riverside', 'Santa Cruz', 'Shasta',
       'San Joaquin', 'Ventura', 'Stanislaus', 'Tulare', 'San Benito',
       'San Luis Obispo', 'San Bernardino', 'Santa Barbara', 'Nevada',
       'Kern', 'Monterey', 'Mendocino', 'Amador', 'Imperial', 'Butte',
       'El Dorado', 'Siskiyou', 'Yuba', 'Unknown', 'Calaveras', 'Merced',
       'Mono', 'Inyo', 'Sutter', 'Colusa', 'Kings', 'Glenn', 'Tuolumne',
       'Alpine', 'Plumas', 'Del Norte', 'Tehama', 'Lake', 'Mariposa',
       'Trinity', 'Sierra', 'Lassen', 'Modoc'], dtype=object)

In [39]:
# Clean data -  remove 'unknown'
clean_covid_df = covid_cali_df[covid_cali_df.county != 'Unknown']
clean_covid_df.head()

,date,county,state,fips,cases,deaths
5,2020-01-25,Orange,California,6059.0,1,0.0
9,2020-01-26,Los Angeles,California,6037.0,1,0.0
10,2020-01-26,Orange,California,6059.0,1,0.0
14,2020-01-27,Los Angeles,California,6037.0,1,0.0
15,2020-01-27,Orange,California,6059.0,1,0.0


In [40]:
# # Confirm that only 58 counties in California
clean_covid_df.county.nunique()

58

In [41]:
covid_cali_2020_df = clean_covid_df.loc[clean_covid_df["date"] < "2021-01-01"]
covid_cali_2020_df.head()

,date,county,state,fips,cases,deaths
5,2020-01-25,Orange,California,6059.0,1,0.0
9,2020-01-26,Los Angeles,California,6037.0,1,0.0
10,2020-01-26,Orange,California,6059.0,1,0.0
14,2020-01-27,Los Angeles,California,6037.0,1,0.0
15,2020-01-27,Orange,California,6059.0,1,0.0


In [42]:
# Identify date total nunique() total
covid_cali_2020_df['date'].nunique()

342

In [43]:
# Identify earliest date
covid_cali_2020_df['date'].min()

Timestamp('2020-01-25 00:00:00')

In [44]:
# Identify max() date to confirm it reflects 12-31-2020
covid_cali_2020_df['date'].max()

Timestamp('2020-12-31 00:00:00')

## Time

In [45]:
# import time
# import datetime

# date_time = covid_cali_2020_df['date']
# date_time_format = "%Y-%m-%d %H:%M:%S"
# date_time

# time_object = time.strptime(date_time, date_time_format)
# epoch_timestamp = time.mktime(time_object)
# epoch_timestamp

In [46]:
# import time
# import datetime
# d = datetime.date(covid_cali_2020_df['date'])

# covid_cali_2020_df['date'] = time.mktime(d.timetuple())
# covid_cali_2020_df

## Create DataFrames for specific viualization needs
- cali only DataFrames grouped by date with cases total 
- socal only DataFrames grouped by county with cases total
- nocal only DataFrames grouped by county wth cases total

In [47]:
# Get clean geo data
new_geo_ca_df=geo_ca_df[["GEOID", "NAME","INTPTLAT","INTPTLON"]]
new_geo_ca_df.head()

,GEOID,NAME,INTPTLAT,INTPTLON
0,6027,Inyo,36.561977,-117.403927
1,6013,Contra Costa,37.919479,-121.951543
2,6003,Alpine,38.621783,-119.798352
3,6023,Humboldt,40.706655,-123.926176
4,6101,Sutter,39.036190,-121.703940


In [48]:
clean_geo_ca_df=new_geo_ca_df.rename(columns={"GEOID": "fips",
                                           "NAME": "county",
                                          "INTPTLAT":"latitude",
                                          "INTPTLON":"longitude"
                                         })
clean_geo_ca_df.head()

,fips,county,latitude,longitude
0,6027,Inyo,36.561977,-117.403927
1,6013,Contra Costa,37.919479,-121.951543
2,6003,Alpine,38.621783,-119.798352
3,6023,Humboldt,40.706655,-123.926176
4,6101,Sutter,39.036190,-121.703940


In [49]:
clean_geo_ca_df.nunique()

fips         58
county       58
latitude     58
longitude    58
dtype: int64

In [50]:
covid_cali_2020_df.dtypes

date      datetime64[ns]
county            object
state             object
fips             float64
cases              int64
deaths           float64
dtype: object

In [51]:
# group all of CA data by date
cali_groupby_date_cases_df = covid_cali_2020_df.groupby('date')

In [52]:
# Get sum for cases by date
cali_date_total = cali_groupby_date_cases_df['cases'].sum()
# Confirm data is correct
cali_date_total.head()

date
2020-01-25    1
2020-01-26    2
2020-01-27    2
2020-01-28    2
2020-01-29    2
Name: cases, dtype: int64

In [53]:
# Create series into DataFrame
cali_date_total_df = pd.DataFrame(cali_date_total)
cali_date_total_df.head()

,cases
date,
2020-01-25,1
2020-01-26,2
2020-01-27,2
2020-01-28,2
2020-01-29,2


In [54]:
# Identify total day count
cali_date_total_df.count()

cases    342
dtype: int64

In [55]:
# Create socalArray from dataFrame with socal counites only
socalArray = ["Inyo", "Kern", "San Luis Obispo", "Santa Barbara", "Ventura", "Los Angeles", "San Bernardino", "Orange", "Riverside", "San Diego", "Imperial"]

# Crating only socal_data_df by selecting cocal counties only
socal_data_df = covid_cali_2020_df[covid_cali_2020_df["county"].isin(socalArray)]
socal_data_df.head()

,date,county,state,fips,cases,deaths
5,2020-01-25,Orange,California,6059.0,1,0.0
9,2020-01-26,Los Angeles,California,6037.0,1,0.0
10,2020-01-26,Orange,California,6059.0,1,0.0
14,2020-01-27,Los Angeles,California,6037.0,1,0.0
15,2020-01-27,Orange,California,6059.0,1,0.0


In [56]:
# Confirm that only the 11 socal counites are in dataFrame
socal_data_df.nunique()

date       342
county      11
state        1
fips        11
cases     2633
deaths    1040
dtype: int64

In [57]:
# Grouped socal by county
grouped_socal_counties = socal_data_df.groupby('county')

In [58]:
# Get sum for cases by county
grouped_socal_county_totals =  grouped_socal_counties['cases'].sum()
grouped_socal_county_totals.head()

county
Imperial        2486469
Inyo              39360
Kern            6042885
Los Angeles    59976957
Orange         11731923
Name: cases, dtype: int64

In [59]:
grouped_socal_county_totals_df = pd.DataFrame(grouped_socal_county_totals)
grouped_socal_county_totals_df

,cases
county,
Imperial,2486469
Inyo,39360
Kern,6042885
Los Angeles,59976957
Orange,11731923
Riverside,12947120
San Bernardino,12451039
San Diego,10827207
San Luis Obispo,768331


In [60]:
merge_socal_geo_df = pd.merge(grouped_socal_county_totals_df,clean_geo_ca_df, on="county", how="inner")
merge_socal_geo_df

,county,cases,fips,latitude,longitude
0,Imperial,2486469,6025,33.040814,-115.355400
1,Inyo,39360,6027,36.561977,-117.403927
2,Kern,6042885,6029,35.346629,-118.729506
3,Los Angeles,59976957,6037,34.196398,-118.261862
4,Orange,11731923,6059,33.675687,-117.777207
5,Riverside,12947120,6065,33.729827,-116.002239
6,San Bernardino,12451039,6071,34.857220,-116.181197
7,San Diego,10827207,6073,33.023604,-116.776117
8,San Luis Obispo,768331,6079,35.385224,-120.447545
9,Santa Barbara,1801476,6083,34.537057,-120.039973


In [61]:
merge_socal_geo_df.dtypes

county        object
cases          int64
fips           int64
latitude     float64
longitude    float64
dtype: object

In [62]:
# Using socal_data_df, merge with clean_covid_cali_df and create duplicates for socal counties
duplicates = pd.merge(clean_covid_df, socal_data_df, how= 'inner', left_on=['date', 'county'], right_on=['date', 'county'], left_index=True)

# Use .drop duplicates on index to create nocal_data_df
norcal_data_df = clean_covid_df.drop(duplicates.index)
norcal_data_df.head()

,date,county,state,fips,cases,deaths
36,2020-01-31,Santa Clara,California,6085.0,1,0.0
42,2020-02-01,Santa Clara,California,6085.0,1,0.0
49,2020-02-02,San Francisco,California,6075.0,2,0.0
50,2020-02-02,Santa Clara,California,6085.0,2,0.0
57,2020-02-03,San Francisco,California,6075.0,2,0.0


In [63]:
# Confirm that only 47 counties for nocal
norcal_data_df.nunique()

date       359
county      58
state        1
fips        58
cases     5935
deaths     743
dtype: int64

In [64]:
# Grouped norcal data by county
grouped_norcal_county_df = norcal_data_df.groupby('county')

In [65]:
# Get cases sum() by county 
grouped_norcal_county_total = grouped_norcal_county_df['cases'].sum()
grouped_norcal_county_total.head()

county
Alameda      5741871
Alpine          4436
Amador        150719
Butte         716705
Calaveras      99176
Name: cases, dtype: int64

In [66]:
# Create DataFrame from series
grouped_norcal_county_total_df = pd.DataFrame(grouped_norcal_county_total)
grouped_norcal_county_total_df.head()

,cases
county,
Alameda,5741871
Alpine,4436
Amador,150719
Butte,716705
Calaveras,99176


In [67]:
merge_norcal_geo_df = pd.merge(grouped_norcal_county_total_df,clean_geo_ca_df, on="county", how="inner")
merge_norcal_geo_df.head()

,county,cases,fips,latitude,longitude
0,Alameda,5741871,6001,37.647139,-121.912488
1,Alpine,4436,6003,38.621783,-119.798352
2,Amador,150719,6005,38.443550,-120.653856
3,Butte,716705,6007,39.665959,-121.601919
4,Calaveras,99176,6009,38.183900,-120.561442


## Make connection to Postgres

In [68]:
clean_covid_df.dtypes

date      datetime64[ns]
county            object
state             object
fips             float64
cases              int64
deaths           float64
dtype: object

In [69]:
# Make connection to telecommunication_db in postgresql
connection_string = f"postgres:{pg_password}@localhost:5432/covid_mask_effect_db"
engine = create_engine(f'postgresql://{connection_string}')

In [70]:
# Read in tables_names() from postgres
engine.table_names()

['cali_2020', 'socal_geo', 'norcal_geo']

## Load DataFrames to Postgres

In [71]:
#cali_date_total_df.to_sql(name='cali_2020', con=engine, if_exists='append', index=True)

In [73]:
#merge_socal_geo_df.to_sql(name='socal_geo', con=engine, if_exists='append', index=False)

In [74]:
#merge_norcal_geo_df.to_sql(name='norcal_geo', con=engine, if_exists='append', index=False)

## Read in tables from pgadmin to confirm data was received correclty

In [75]:
pd.read_sql_query('SELECT * FROM cali_2020', con=engine).head()

,date,cases
0,2020-01-25,1
1,2020-01-26,2
2,2020-01-27,2
3,2020-01-28,2
4,2020-01-29,2


In [76]:
pd.read_sql_query('SELECT * FROM socal_geo', con=engine).head()

,county,cases,fips,latitude,longitude
0,Imperial,2486469,6025,33.040814,-115.355400
1,Inyo,39360,6027,36.561977,-117.403927
2,Kern,6042885,6029,35.346629,-118.729506
3,Los Angeles,59976957,6037,34.196398,-118.261862
4,Orange,11731923,6059,33.675687,-117.777207


In [77]:
pd.read_sql_query('SELECT * FROM norcal_geo', con=engine).head()

,county,cases,fips,latitude,longitude
0,Alameda,5741871,6001,37.647139,-121.912488
1,Alpine,4436,6003,38.621783,-119.798352
2,Amador,150719,6005,38.443550,-120.653856
3,Butte,716705,6007,39.665959,-121.601919
4,Calaveras,99176,6009,38.183900,-120.561442


# Convert new tables into csv

In [78]:
cali_date_total_df.to_csv('output/cali_date_total_df.csv')

In [79]:
merge_socal_geo_df.to_csv('output/merge_socal_geo_df.csv')

In [80]:
merge_norcal_geo_df.to_csv('output/merge_norcal_geo_df.csv')

In [81]:
covid_cali_2020_df.to_csv('output/covid_cali_2020_df.csv')